### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv('.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
request_url =  (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
# Create an empty list to store the reviews 
# (reviews_list NOT list_review) (reviews NOT results)
reviews_list = []

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    page_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
    )
    page_url = f"{page_url}&page={str(page)}"

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Checked page {page}")
        # Print the page number that had no results then break from the loop
    except:
        print(f"Page Number with no results {page}")

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [5]:
# Preview the first 5 results in JSON format (reviews_list NOT list_review)
print(reviews_list[:5])
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list, indent=4))

[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}, 'word_count': 29

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

all_results_df = pd.json_normalize(reviews_list)
all_results_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.  
# mine to help understand i.e.,"main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
# End string should include "Review" to avoid cutting title early

# I looked to W3 and pandas.pydata to see about a str.extract solution that will
# look at only the headline.main then extract or capture anything between \u2018 
# and \u2019 and then anything after "Review"
# then I added it as a new column at the end call "title"

# all_results_df['title'] = all_results_df["headline.main"].str.extract(r"\u2018(.*)\u2019*Review")
# all_results_df.head()

# oh there was a hint: 
all_results_df['title'] = all_results_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
all_results_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string

all_results_df['keywords'] = all_results_df['keywords'].apply(extract_keywords)
all_results_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles = all_results_df['title'].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
        # According to the webside:
        # While our legacy rate limits have been disabled for some time, we do still 
        # have some upper limits to help mitigate needlessly high bulk scraping. They 
        # sit somewhere in the 50 requests per second range. This limit could change 
        # at any time so be respectful of the service we have built and respect the 
        # 429 if you receive one.
    
    # Add 1 to the request counter
    request_counter = request_counter + 1

    if (request_counter % 50 == 0):
        print('Application is Zzzzzzzzzz')
        time.sleep(1)

    # Perform a "GET" request for The Movie Database
    movie_url = url + title + tmdb_key_string
    results = requests.get(movie_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = results["results"][0]["id"]
        # Make a request for a the full movie details
        movie_org_url = "https://api.themoviedb.org/3/movie/"
        movie_url = f"{movie_org_url}{movie_id}?{tmdb_key_string}" 
        
        # Execute "GET" request with url
        movie_results = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_results.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages= [language['name'] for language in 
                                movie_results.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in 
                                    movie_results.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        # From instuctions title, original_title, budget, original_language, homepage, overview, 
        # popularity, runtime, revenue, release_date, vote_average, vote_count, 
        # as well as the genres, spoken_languages, and production_countries 
        movie_info = {
            'title': title,
            'Original Title': movie_results.get('original_title'),
            'Budget': movie_results.get('budget'),
            'Original Language': movie_results.get('original_language'),
            'Homepage': movie_results.get('homepage'),
            'Overview': movie_results.get('overview'),
            'Popularity: So popular': movie_results.get('popularity'),
            'Runtime': movie_results.get('runtime'),
            'Revenue': movie_results.get('revenue'),
            'Release Date': movie_results.get('release_date'),
            'Average Votes': movie_results.get('vote_average'),
            'Counts of Votes': movie_results.get('vote_count'),
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        tmdb_movies_list.append(movie_info)
        # Print out the title that was found        
        print(f"Found details for {title}")
    except:
        print(f"No details found for {title}")
        # Print out the title that was found
print(f"We have {len(tmdb_movies_list)} movies!")

Found details for The Attachment Diaries
No details found for What’s Love Got to Do With It?’ Probably a Lo
Found details for You Can Live Forever
Found details for A Tourist’s Guide to Love
Found details for Other People’s Children
Found details for One True Loves
Found details for The Lost Weekend: A Love Story
Found details for A Thousand and One
Found details for Your Place or Mine
Found details for Love in the Time of Fentanyl
Found details for Pamela, a Love Story
Found details for In From the Side
Found details for After Love
Found details for Alcarràs
Found details for Nelly & Nadine
Found details for Lady Chatterley’s Lover
Found details for The Sound of Christmas
Found details for The Inspection
Found details for Bones and All
Found details for My Policeman
Found details for About Fate
Found details for Waiting for Bojangles
Found details for I Love My Dad
Found details for A Love Song
Found details for Alone Together
Found details for Art of Love
Found details for The Wheel


In [12]:
# Preview the first 5 results in JSON format
print(tmdb_movies_list[:5])
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[{'title': 'The Attachment Diaries', 'Original Title': 'El apego', 'Budget': 0, 'Original Language': 'es', 'Homepage': '', 'Overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'Popularity: So popular': 1.186, 'Runtime': 102, 'Revenue': 0, 'Release Date': '2021-10-07', 'Average Votes': 4.0, 'Counts of Votes': 2, 'genres': ['Drama', 'Mystery', 'Thriller', 'Horror'], 'spoken_languages': ['Español'], 'production_countries': ['Argentina']}, {'title': 'You Can Live Forever', 'Original Title': 'You Can Live Forever', 'Budget': 0, 'Original Language': 'en', 'Homepage': 'https://gooddeedentertainment.com/you-can-live-forever/', 'Overview': "When Jaim

In [13]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list) 
tmdb_df  = pd.json_normalize(tmdb_movies_list) 
tmdb_df 

,title,Original Title,Budget,Original Language,Homepage,Overview,Popularity: So popular,Runtime,Revenue,Release Date,Average Votes,Counts of Votes,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.186,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Español],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",4.533,96,15055,2023-03-24,6.963,27,"[Drama, Romance]","[English, Français]",[Canada]
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.075,96,0,2023-04-21,6.365,144,"[Romance, Comedy]","[English, Tiếng Việt]",[United States of America]
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.866,104,84178,2022-09-21,6.915,164,"[Drama, Comedy]","[English, Français]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,8.979,100,37820,2023-04-07,6.700,54,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",12.376,130,0,2017-01-11,5.614,185,[Drama],"[English, Português]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,7.647,83,0,2017-01-14,6.059,111,[Comedy],[Français],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.691,103,0,2016-12-02,6.159,22,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.015,111,413844,2017-03-10,6.173,240,"[Romance, Drama, War]","[Türkçe, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title

movies_merged_df = pd.merge(tmdb_df, all_results_df, on="title", how="inner")
movies_merged_df
movies_merged_df.to_csv("output/movies_merged_df.csv", index=False)

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing full_movie_data
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]","'"]

# Loop through the list of columns to fix columns_to_fix

for columns in columns_to_fix :
    # Convert the column to type 'str'  astype()
    movies_merged_df[columns] = movies_merged_df[columns].astype(str)

    # Loop through characters to remove characters_to_remove str.replace()
    for char in characters_to_remove:
        movies_merged_df[columns] = movies_merged_df[columns].str.replace(char, '')

# Display the fixed DataFrame
movies_merged_df.head(3)

,title,Original Title,Budget,Original Language,Homepage,Overview,Popularity: So popular,Runtime,Revenue,Release Date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.186,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",4.533,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.075,96,0,2023-04-21,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None


In [16]:
# Drop "byline.person" column
movies_dropped_byline=movies_merged_df.drop(['byline.person'], axis=1)
movies_dropped_byline.head(3)

,title,Original Title,Budget,Original Language,Homepage,Overview,Popularity: So popular,Runtime,Revenue,Release Date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.186,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",4.533,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.075,96,0,2023-04-21,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None


In [17]:
# Delete duplicate rows and reset index
final_movie_data = movies_dropped_byline.drop_duplicates().reset_index(drop=True)
final_movie_data.head(10)

,title,Original Title,Budget,Original Language,Homepage,Overview,Popularity: So popular,Runtime,Revenue,Release Date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.186,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",4.533,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.075,96,0,2023-04-21,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.866,104,84178,2022-09-21,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,8.979,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.920,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
6,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,24.295,116,3395595,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
7,Your Place or Mine,Your Place or Mine,0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,21.333,109,0,2023-02-10,...,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,,An intimate portrait of a community fighting t...,0.817,85,0,2023-02-03,...,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None
9,"Pamela, a Love Story","Pamela, A Love Story",0,en,https://www.netflix.com/title/81590934,"In her own words, through personal video and d...",11.294,113,0,2023-01-30,...,295,"‘Pamela, a Love Story’ Review: A Frank Look Back",None,None,"Pamela, a Love Story",None,None,None,By Glenn Kenny,None


In [18]:
# Export data to CSV without the index
final_movie_data.to_csv("output/movies_output.csv", index=False)